In [50]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [75]:
#Read in data from CSV files.
df = pd.read_csv('raw_data/PO_Dataset.csv')
name_mapping = pd.read_csv('clean_data/Clean_Code_Master_list.csv')
df.head()


,Company #,Purchase Order,Item,Vendor,Description,Unit of Measure,Units,Unit Cost,Cost,Cost Code
0,8,1200-001,1,Paragon Electrical Installations Ltd.,Additional smoke detector/re-verification,LS,0,0.0,1444.0,26-20-20
1,8,1200-002,1,Accurate Aluminum Ltd,S&I railing as per quote Aug. 13 2015,LS,0,0.0,500.0,05-52-20
2,8,1200-003,1,Dura Productions,S&I metal ramp,LS,0,0.0,795.0,05-52-20
3,8,1200-004,1,Friesen Floors & Window Fashions Ltd,S&I hardwood flooring for enclosed balcony area,LS,0,0.0,2314.0,09-64-33
4,8,1209-1-01,1,Alba Painting Ltd.,Painting of two offices,LS,0,0.0,900.0,09-91-40


In [52]:
#The Convert the Units column to float
df['Units'] = pd.to_numeric(df['Units'], errors='coerce').fillna(0)
df['Units'] = df['Units'].astype('float64')

#Drop lines with null values
df.dropna(inplace=True)

#Read in Master list of valid cost codes
df_ml = pd.read_csv('raw_data/Code_Master_list.csv')

#Drop rows where the cost code is not in the master list
df = df[df['Cost Code'].isin(df_ml['Cost Code'])].dropna()

#Create a new dataframe that takes only the 90th quartile of data from the 3 numerical columns.
df_90 = df[df['Cost'] < df['Cost'].quantile(.90)]
df_90 = df_90[df_90['Units'] < df_90['Units'].quantile(.90)]
df_90 = df_90[df_90['Unit Cost'] < df_90['Unit Cost'].quantile(.90)]

# It's a good practice to scale numerical data
# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() 
numerical = ['Units','Unit Cost','Cost']

df_90[numerical] = scaler.fit_transform(df_90[numerical])

# When splitting for training and testing later, we'll need a minimum of 2 examples of each cost code.
# Assign cost code to a variable
df_count = df_90['Cost Code'].value_counts()

#New dataframe only includes lines with cost codes with a count of 2 or greater
df_90 = df_90[~df_90['Cost Code'].isin(df_count[df_count < 2].index)]


#One Hot Encode categorical features
categorical = ['Vendor', 'Unit of Measure']
df_90 = pd.get_dummies(df_90, columns = categorical )

#Numerically encode cost codes.
le = LabelEncoder()
cost_code = df_90['Cost Code']
df_90['Cost Code Encoded'] = le.fit_transform(cost_code)

#drop features I won't be using
df_90 = df_90.drop(['Company #','Purchase Order', 'Item'], axis = 1)

df = df_90


In [53]:
cost_codes = df['Cost Code Encoded']
features = df.drop(['Cost Code','Cost Code Encoded'], axis=1)


In [54]:
#Use sklearn train test split to split the data into training and testing sets. 
#Testing set is 20% of total dataset size.
#Stratify the data so we don't introduce bias in the sets.

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    cost_codes,
                                                    test_size = 0.2,
                                                    stratify = cost_codes
                                                   )



In [55]:
print(X_train.shape)
print(y_train.shape)

(22739, 487)
(22739,)


In [56]:
#Split out X_train and X_test Descriptions for use in sepearate model.
X_train_desc = X_train['Description'].copy()
X_train = X_train.drop('Description', axis=1)

X_test_desc = X_test['Description'].copy()
X_test = X_test.drop('Description', axis=1)

In [57]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter = 5, random_state=42, tol = 1e-3)),
               ])
sgd.fit(X_train_desc, y_train)


y_pred = sgd.predict(X_test_desc)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.43746701846965697
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.71      0.95      0.81       112
           3       0.62      0.62      0.62         8
           4       0.89      0.59      0.71        29
           5       0.65      0.79      0.71        19
           6       0.62      1.00      0.77         5
           7       0.62      0.43      0.51        23
           8       0.00      0.00      0.00         0
           9       0.40      0.80      0.53         5
          10       0.00      0.00      0.00         1
          11       0.52      0.33      0.40        46
          12       0.55      0.86      0.67       156
          13       0.00      0.00      0.00         1
          14       0.65      0.45      0.53        29
          15       0.00      0.00      0.00         2
          16       0.22      0.17      0.19        12
          17       0.47      0.16      0.24        5

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\li

In [73]:
df_output = pd.DataFrame({'Text':X_test_desc ,'Prediction':le.inverse_transform(y_pred), 'Actual':le.inverse_transform(y_test)})

In [99]:
name_mapping.head()

,Cost Code,Description
0,00-10-17,geotechnical consultant
1,00-61-13,subtrade bonds
2,01-30-12,project manager
3,01-30-14,project coordinator
4,01-30-23,finishing superintendent


In [118]:
out = df_output.merge(name_mapping, left_on='Actual',right_on='Cost Code')
out.rename(index=str, columns={'Description':'Actual Desc.'}, inplace = True)
out.drop(['Cost Code'], axis = 1, inplace = True)
out.head()

,Text,Prediction,Actual,Actual Desc.
0,18.5L Demineralized water,01-51-36,01-51-36,temporary water
1,Nozzle red fog,01-52-23,01-51-36,temporary water
2,9OZ cup sleeves,01-57-40,01-51-36,temporary water
3,Monthly August,06-46-33,01-51-36,temporary water
4,OH/P 15%,01-30-31,01-51-36,temporary water


In [119]:
out = out.merge(name_mapping, left_on='Prediction',right_on='Cost Code')
out.drop(['Cost Code'], axis = 1, inplace=True)
out.rename(index=str, columns={'Description':'Predicted Desc.'}, inplace=True)
out.head()

,Text,Prediction,Actual,Actual Desc.,Predicted Desc.
0,18.5L Demineralized water,01-51-36,01-51-36,temporary water,temporary water
1,18.5L Demineralized water,01-51-36,01-51-36,temporary water,temporary water
2,fuel cost,01-51-36,03-31-42,concrete materials - below-grade horizontals,temporary water
3,Fuel Cost,01-51-36,03-31-43,concrete material - above grade verticals,temporary water
4,Fuel Cost,01-51-36,03-31-43,concrete material - above grade verticals,temporary water


In [120]:
cols = ['Text', 'Prediction','Predicted Desc.', 'Actual', 'Actual Desc.']
out = out[cols]

In [122]:
out.head()

,Text,Prediction,Predicted Desc.,Actual,Actual Desc.
0,18.5L Demineralized water,01-51-36,temporary water,01-51-36,temporary water
1,18.5L Demineralized water,01-51-36,temporary water,01-51-36,temporary water
2,fuel cost,01-51-36,temporary water,03-31-42,concrete materials - below-grade horizontals
3,Fuel Cost,01-51-36,temporary water,03-31-43,concrete material - above grade verticals
4,Fuel Cost,01-51-36,temporary water,03-31-43,concrete material - above grade verticals
